# Step 1: Setup and Initialization
First, make sure you have the Earth Engine Python API installed and authenticated on your local machine. You can install it via pip and authenticate using the following commands:

In [48]:
# !pip3 install earthengine-api
# !pip3 install folium
# earthengine authenticate
# !pip3 install geemap

In [49]:
# ALL
import ee
import folium
import geemap
import PIL
import urllib

# AS
import matplotlib.pyplot as plt
import numpy as np

# FROM
from IPython.display import Image

# Step 2: Define Area of Interest
Specify the geographic boundaries of your area of interest (AOI) in Sudan. This could be a specific region where you suspect crop diseases. The coordinates are defined in longitude and latitude.

In [50]:
# Authenticate the Earth Engine module.
ee.Authenticate()

True

In [51]:
# Initialize the Earth Engine library
ee.Initialize()

# Step 2: Define Area of Interest
Specify the geographic boundaries of your area of interest (AOI) in Sudan. This could be a specific region where you suspect crop diseases. The coordinates are defined in longitude and latitude.

In [52]:
# Define a Point and then buffer it to create an area (adjust buffer size as needed).
roi = ee.Geometry.Polygon([
    [
        [32.40230249855585, 15.312285731822914],
        [32.40230249855585, 13.546161854195942],
        [33.72066187355585, 13.546161854195942],
        [33.72066187355585, 15.312285731822914]
    ]
])

# Define Sentinel-2 image collection
sentinelCollection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
imageCollection = ee.ImageCollection("COPERNICUS/S2_SR") # ee.ImageCollection('COPERNICUS/S2')  # Assuming Sentinel-2

# Step 3: Acquire Sentinel-2 Imagery
Fetch Sentinel-2 imagery that covers your AOI. Filter by date and other relevant properties like cloud cover.

In [53]:
# Set variables
start_date_s = ee.String('2021-02-01') # we use the string at the raster name
end_date_s = ee.String('2022-05-20') # we use the string at the raster name
cloud_percentage = ee.Number(30)  # format : %30, %50 etc. Lower threshold is recommended.
interval = ee.Number(10) # settings for the years to filter on
increment = 'day' # settings for the years to filter on

In [54]:
# Calculated variables
start_date = ee.Date(start_date_s) # converting string to date data type
end_date = ee.Date(end_date_s)

In [55]:
# Define the region of interest, assuming 'roi' is predefined
Map = geemap.Map()

In [56]:
# Define the region of interest (roi) somehow, here we assume it's predefined
# roi = ee.FeatureCollection('path_to_your_roi')

In [57]:
# Center the map on the region of interest
# Map.centerObject(roi, 8)  # This line can only be replicated in a GEE JavaScript environment

In [58]:
# make a list of start years
second_date = start_date.advance(interval, increment).millis()
increase = second_date.subtract(start_date.millis())
list = ee.List.sequence(start_date.millis(), end_date.millis(), increase)

In [59]:
# Get the number of images
len(list.getInfo())

48

In [60]:
# Function to add NDVI
def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [61]:
# Load Sentinel-2 imagery and filter by date, cloud cover and area
# Get the image collection
sentinel_ts = ee.ImageCollection(imageCollection
  .filterBounds(roi)
  .filterDate(start_date, end_date)
  .select(['B4', 'B8'])
  .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))
  .map(addNDVI)
  .select(['NDVI']))

# Step 4: Preprocess the Data
Select specific bands and perform any necessary preprocessing like normalization or spectral indices (e.g., NDVI for vegetation health).

In [62]:
# Monthly composite
def monthly_composite(date):
    date = ee.Date(date)
    return sentinel_ts.select('NDVI').filterDate(date, date.advance(interval, increment)).mean().set('system:time_start', date.millis())

In [63]:
# Create a new collection by mapping the composite function over the date list
new_col = ee.ImageCollection.fromImages(list.map(monthly_composite))

# Clip the collection to the region of interest
new_col = new_col.toBands().clip(roi)

In [64]:
# Available band names
band_names = new_col.bandNames().getInfo()
print("Available band names:", band_names)

Available band names: ['0_NDVI', '1_NDVI', '2_NDVI', '3_NDVI', '4_NDVI', '5_NDVI', '6_NDVI', '7_NDVI', '8_NDVI', '9_NDVI', '10_NDVI', '11_NDVI', '12_NDVI', '13_NDVI', '14_NDVI', '15_NDVI', '16_NDVI', '18_NDVI', '19_NDVI', '20_NDVI', '21_NDVI', '22_NDVI', '23_NDVI', '24_NDVI', '25_NDVI', '26_NDVI', '27_NDVI', '28_NDVI', '29_NDVI', '30_NDVI', '31_NDVI', '32_NDVI', '33_NDVI', '34_NDVI', '35_NDVI', '36_NDVI', '37_NDVI', '38_NDVI', '39_NDVI', '40_NDVI', '41_NDVI', '42_NDVI', '43_NDVI', '44_NDVI', '45_NDVI', '46_NDVI', '47_NDVI']


# Step 5: Visualization
Visualize the data to manually inspect the area for signs of crop disease or to validate outputs.

In [67]:
# Generate a thumbnail URL (not directly possible in Python without using a service URL)
thumbnail = new_col.getThumbURL({
    'min': 0,
    'max': 1,
    'bands': ["0_NDVI", "1_NDVI", "3_NDVI"],
    'dimensions': 1200,
    'region': roi,
})

# Display thumbnail
print('Rectangle region and palette:', thumbnail)

Rectangle region and palette: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/9eca66c11c2c64b849a7f94dc5029f5c-410b98ff52f78f39e75d29a1e5faab39:getPixels


In [79]:
# Convert the image to RGB format
# image_rgb = np.array(PIL.Image.open(urllib.request.urlopen(thumbnail)).convert('RGB'))

# Display the RGB image
# plt.imshow(image_rgb)

In [78]:
# save image PIL.Image.open(urllib.request.urlopen(thumbnail)).convert('RGB')
PIL.Image.open(urllib.request.urlopen(thumbnail)).convert('RGB').save('output.png')

# Multiple

In [125]:
# Define the increment (5 minutes)
delta = 5

# Loop over each interval
current_date = start_date

# Add one day
next_date = ee.Date(current_date).advance(1, 'day')

# Convert ee.Date back to string
next_date_str = next_date.format('YYYY-MM-dd').getInfo()  # Using getInfo() to retrieve the value from the server


while ee.Date(current_date).millis().getInfo() <= ee.Date(end_date).millis().getInfo():
    print('Start')
    
    # Calculate the end date of the current interval
    interval_end = ee.Date(current_date).advance(delta, 'day')
    
    # Format the current date as a string for use in file naming
    current_date_str = ee.Date(current_date).format('YYYY-MM-dd_HHmm').getInfo()
    
    print('Start Sentinal Image Collection')
    
    # Get the image collection for the current interval
    sentinel_ts = ee.ImageCollection(imageCollection
                    .filterBounds(roi)
                    .filterDate(current_date, interval_end)
                    .select(['B4', 'B8'])
                    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))
                    .map(addNDVI)
                    .select(['NDVI'])
                    .first())
    
    print('Processing:')

    # Start the export task
    task = ee.batch.Export.image.toDrive(
        image=sentinel_ts,
        description='imageToDrive',
        folder='GEE_Folder',
        fileNamePrefix='srtm90_v4',
        scale=30,
        region=roi.getInfo()['coordinates'],
        fileFormat='GeoTIFF',
        maxPixels=1e9  # Increase max pixels if needed
    )
    print('Exporting')
    
    task.start()
    print('Task Started')
    
    # Check the status of the export task after a while
    status = task.status()
    print(status)

    # Move to the next interval by updating the current_date
    current_date = interval_end

print("All tasks started.")

Start
Start Sentinal Image Collection
Processing:
Exporting
Task Started
{'state': 'READY', 'description': 'imageToDrive', 'priority': 100, 'creation_timestamp_ms': 1715669591442, 'update_timestamp_ms': 1715669591442, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'RAVASSEKHMKG5FMLOVDYMAJL', 'name': 'projects/earthengine-legacy/operations/RAVASSEKHMKG5FMLOVDYMAJL'}
Start
Start Sentinal Image Collection
Processing:
Exporting
Task Started
{'state': 'READY', 'description': 'imageToDrive', 'priority': 100, 'creation_timestamp_ms': 1715669592496, 'update_timestamp_ms': 1715669592496, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'JVL5V7QEA4VTXFXSWGXW7TJR', 'name': 'projects/earthengine-legacy/operations/JVL5V7QEA4VTXFXSWGXW7TJR'}
Start
Start Sentinal Image Collection
Processing:
Exporting
Task Started
{'state': 'READY', 'description': 'imageToDrive', 'priority': 100, 'creation_timestamp_ms': 1715669593529, 'update_timestamp_ms': 1715669593529, 'start_timestamp_ms'